In [1]:
from qat.lang.AQASM import Program, QRoutine, RZ, RX, CNOT, H, X, Z
from qat.core import Observable, Term
from qat.qpus import get_default_qpu, PyLinalg
from qat.plugins import ScipyMinimizePlugin
from math import sqrt

from pysat.formula import CNF

## SAT

<!-- Descrever 3-SAT -->
O Problema de Satisfatibilidade Booleana é o problema de determinar se existe uma interpretação que satisfaça uma fórmula Booleana. Ou seja, o problema busca uma sequência de valores VERDADEIRO ou FALSO que possa substituir as variáveis da fórmula Booleana de forma que seu valor final seja VERDADEIRO. Caso exista essa sequência, a fórmula é *satisfazível*. Por outro lado, se não existe essa sequência, a expressão definida pela fórmula é FALSO para todos os valores e é *insatisfazível*.


<!-- Descrever CNF -->
## Forma Normal Conjuntiva

Uma fórmula booleana é formada por variáveis booleanas, também chamadas de literais. A Forma Normal Conjuntiva é uma forma padrão que pode ser usada para representar qualquer sentença lógica. Nessa forma, a sentença lógica é representada por uma conjunção de cláusulas. Uma conjunção ($\wedge$) é representada pela operação lógica E. Uma cláusula é uma disjunção ($\vee$, operação lógica OU) de literais. Por exemplo, a expressão

$$ (A \vee B \vee C) \wedge (D \vee E) \wedge (F \vee G)$$

É uma conjunção das cláusulas "A ou B ou C", "D ou E", "F ou G". A FNC é fácil de se trabalhar e manipular e é possível transformar qualquer sentença lógica em uma forma normal conjuntiva aplicando-se regras de inferência e algumas transformações.

Em uma formulação do problema 3-SAT, todas as cláusulas possuem 3 literais. Por exemplo,
$$ (x_1 \vee x_2 \vee x_3) \wedge (\neg x_1 \vee x_2 \vee x_4) \wedge (x_1 \vee \neg x_3 \vee x_4) \wedge (\neg x_2 \vee \neg x_3 \vee x_5)$$
representa uma instância 3-SAT.

<!-- Usando Grover em 3-SAT -->
Grover aplicado ao 3-SAT possui complexidade $O(2^{\frac{n}{2}}) = O(1.414^n)$.
Entretanto, para o problema do 3-SAT, o melhor algoritmo clássico conhecido possui a complexidade de $O(1.307^n)$.
Isso é porque problemas NP-completos possuem uma estrutura que pode ser explorada para se obter soluções, enquanto o algoritmo de Grover é ótimo para problemas sem estrutura.
Apesar de não parecer vantajoso usar Grover para resolver 3-SAT, as técnicas usadas aqui podem ser expandidas para casos gerais, como o k-SAT, no qual Grover supera o melhor algoritmo clássico atual.

# Algoritmo de Grover Aplicado

<!-- Descrever Grover -->
O algoritmo de Grover é um algoritmo quântico que pode ser usado para *busca não estruturada*. O Algoritmo possui a complexidade ótima de $O(\sqrt{N})$ onde $N$ é o tamanho total do espaço da busca.
Sua relevância como algoritmo é pelo fato de que uma grande parte de problemas interessantes, diga-se a classe NP, possuem soluções que são difíceis de computar mas são fáceis de verificar. Portanto, Grover oferece um ganho quadrático para qualquer método de força-bruta para resolver essa classe de problemas.

<!-- Grover não é busca em um banco de dados -->
Quando usa-se Grover para resolver um problema, como quebra de uma chave criptográfica, é comum confundir o seu comportamento. O algoritmo não faz consultas a um banco de dados magicamente codificado, o qual precisaria estar armazenado integralmente no circuito quântico. Em vez disso, dada uma função booleana arbitrária $f:\big\{0,1\big\}^n\rightarrow \{0,1\}$ que seja eficientemente computável, cuja expressão matemática é

$$\begin{equation*}
    f(x) = \left\{
  \begin{array}{ll}
    1, & \hbox{se $x$ resolve o problema,} \\
    0, & \hbox{caso contrário.}
  \end{array}
\right.
\end{equation*}$$
e você quer descobrir o valor $x$ para o qual $f(x) = 1$, então, como você consegue computar $f(x)$, você pode rodar $f$ em um computador quântico e usar Grover para encontrar $x$.

Essa função $f(x)$ será codificada no operador $U_f$ do algoritmo de Grover, também chamado de oráculo. Esse operador deve ser capaz de solucionar o problema em questão, por exemplo, uma instância do problema SAT pode ser implementada usando um circuito clássico. Esse mesmo circuito, se convertido para um circuito quântico equivalente, serve como um operador $U_f$.

## Desafio - Resolvendo o problema 3-SAT usando Grover

Crie um circuito quântico que encontre a solução para a expressão booleana abaixo usando o algoritmo de Grover

$$ (x_1 \vee x_2 \vee x_3) \wedge (\neg x_1 \vee x_2 \vee x_3) \wedge (x_1 \vee \neg x_2 \vee x_3) \wedge (\neg x_1 \vee \neg x_2 \vee x_3) \wedge (\neg x_1 \vee x_2 \vee \neg x_3) \wedge (x_1 \vee \neg x_2 \vee \neg x_3) \wedge (\neg x_1 \vee \neg x_2 \vee \neg x_3)$$

In [3]:
# remove one of the clauses
cnf = CNF(from_file='example.cnf')
num_literals = cnf.nv

print(cnf.clauses)
print(num_literals)

[[1, 2, 3], [-1, 2, 3], [1, -2, 3], [-1, -2, 3], [1, -2, -3], [-1, 2, -3], [-1, -2, -3]]
3
